In [1]:
%matplotlib qt5
from process_files import *
from func_target import *

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


In [2]:
iline = 0
line = 'Fe_I_6173'
# calib data
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
tgdir = config['targetplate']['directory']
line = config['line']
settings = [f for f in os.listdir(tgdir) if 'settings' in f]
settings = tgdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
#
linestr = 'Line_' + str(get_line_num(settings, line, iline))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(tgdir+os.sep+line))//3

In [3]:
# darks
dk0 = np.float64(pf.open(config['darks'][line+'/bbi'])[0].data)
dk1 = np.float64(pf.open(config['darks'][line+'/pol1'])[0].data)
dk2 = np.float64(pf.open(config['darks'][line+'/pol2'])[0].data)
# flats
ff_bbi = np.float64(pf.open(config['flats'][line+'/bbi'])[0].data)
ff_bbi = ff_bbi[:,:,np.newaxis]/np.mean(ff_bbi)
ff_pol = np.float64(pf.open(config['flats'][line+'/pol'])[0].data)
ff_pol1 = ff_pol[:,:,0:1,0]
ff_pol2 = ff_pol[:,:,4:5,0]

In [ ]:
# Get target
dc0 = data_cube(tgdir, line, 0, 0)
print('Reading BBI target image from ', dc0.file)
tg0 = np.uint16(16*(dc0.data-dk0)/ff_bbi)
dc1 = data_cube(tgdir, line, 1, 0)
print('Reading POL1 target image from ', dc1.file)
tg1 = np.uint16(16*(dc1.data-dk1)/ff_pol1)
tg1 = np.flipud(tg1)
dc2 = data_cube(tgdir, line, 2, 0)
print('Reading POL2 target image from ', dc2.file)
tg2 = np.uint16(16*(dc2.data-dk2)/ff_pol2)
#
config['targetplate']['flip1_lr'] = 0
config['targetplate']['flip1_ud'] = 1
config['targetplate']['flip2_lr'] = 0
config['targetplate']['flip2_ud'] = 0
config.write()
#

In [ ]:
tg0_ = np.mean(tg0, axis=2)
tg1_ = np.mean(tg1, axis=2)
tg2_ = np.mean(tg2, axis=2)
blink_frames([tg0_, tg1_, tg2_], repeat=1)

In [ ]:
npts=12
plt.imshow(tg0_)
pts0 = plt.ginput(n=npts, timeout=0, mouse_add=None, mouse_pop=None, mouse_stop=None)
plt.imshow(tg1_)
pts1 = plt.ginput(n=npts, timeout=0, mouse_add=None, mouse_pop=None, mouse_stop=None)
plt.imshow(tg2_)
pts2 = plt.ginput(n=npts, timeout=0, mouse_add=None, mouse_pop=None, mouse_stop=None)
pts0_ = np.array(pts0)
pts1_ = np.array(pts1)
pts2_ = np.array(pts2)
aff1, temp = cv2.estimateAffine2D(pts1_, pts0_)
aff2, temp = cv2.estimateAffine2D(pts2_, pts0_)

In [ ]:
dir_tree = tgdir.split(os.sep)
l0dir = os.sep.join(dir_tree[0:-1])+os.sep+'L0'
if not (os.path.exists(l0dir)): os.mkdir(l0dir)
l0dir += os.sep+dir_tree[-1]
if not (os.path.exists(l0dir)): os.mkdir(l0dir)
tg_name = l0dir+os.sep+os.path.split(dc0.file)[-1]
tg_name = tg_name.replace('.DAT', '.fits').replace('bbi', line)
config['targetplate'][line+'/l0data'] = tg_name
config.write()
#
hdu1 = pf.PrimaryHDU(tg0)
hdu2 = pf.ImageHDU(tg1)
hdu3 = pf.ImageHDU(tg2)
hdu4 = pf.ImageHDU(np.array([aff1, aff2]))
hdul = pf.HDUList([hdu1, hdu2, hdu3, hdu4])
hdul.writeto(tg_name, overwrite=True)
print('Target plate data saved as: ', tg_name)

In [4]:
tg_name = config['targetplate'][line+'/l0data'] 
hdul = pf.open(tg_name)
tg0 = hdul[0].data
tg1 = hdul[1].data
tg2 = hdul[2].data
aff = hdul[3].data
aff1 = aff[0]
aff2 = aff[1]
print('Read targetplate data from: ', tg_name)

Read targetplate data from:  /arch/1/VTT/hellride/raw/20230504/L0/Targetplate_1203/HELLRIDE_Fe_I_6173_20230504_120328106_tp.fits


In [5]:
tg0_ = np.mean(tg0, axis=2)
tg1_ = np.mean(tg1, axis=2)
tg2_ = np.mean(tg2, axis=2)
blink_frames([tg0_, tg1_, tg2_], repeat=1)

libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [31]:
sx1, sy1 = np.linalg.norm(aff1[:,0]), np.linalg.norm(aff1[:,1])
sx2, sy2 = np.linalg.norm(aff2[:,0]), np.linalg.norm(aff2[:,1])
mag = np.mean([sx1, sy1, sx2, sy2])
#
rotang1 = -np.degrees(np.arctan2(-aff1[0,1]/sy1, aff1[0,0]/sx1))
rotang2 = -np.degrees(np.arctan2(-aff2[0,1]/sy2, aff2[0,0]/sx2))
#
sh1_ = np.array([-4,-3])
sh2_ = np.array([-5,-2])
sh1 = aff1[::-1,2] + sh1_
sh2 = aff2[::-1,2] + sh2_
#
tg0_rot = tg0_
tg1_rot = rotate(tg1_, rotang1, mode='nearest', reshape=False)
tg2_rot = rotate(tg2_, rotang2, mode='nearest', reshape=False)
#
tg0_mag = tg0_rot
tg1_mag = zoom_clipped(tg1_rot, mag) 
tg2_mag = zoom_clipped(tg2_rot, mag)
#
tg0_shift = tg0_mag
tg1_shift = shift(tg1_mag, sh1, mode='nearest')
tg2_shift = shift(tg2_mag, sh2, mode='nearest')

In [32]:
tg0_plt = (tg0_shift-tg0_shift.min())/(tg0_shift.max()-tg0_shift.min())
tg1_plt = (tg1_shift-tg1_shift.min())/(tg1_shift.max()-tg1_shift.min())
tg2_plt = (tg2_shift-tg2_shift.min())/(tg2_shift.max()-tg2_shift.min())
blink_frames([tg0_plt, tg1_plt], repeat=5)
# plt.figure(); plt.imshow(tg0_shift)
# plt.figure(); plt.imshow(tg1_shift)
# plt.figure(); plt.imshow(tg2_shift)

In [26]:
config['targetplate'][line+'/rotang1'] = rotang1
config['targetplate'][line+'/rotang2'] = rotang2
config['targetplate'][line+'/mag'] = mag
config['targetplate'][line+'/shift1_x'] = sh1[1]
config['targetplate'][line+'/shift1_y'] = sh1[0]
config['targetplate'][line+'/shift2_x'] = sh2[1]
config['targetplate'][line+'/shift2_y'] = sh2[0]
config.write()